In [ ]:
from pace.missingness import *
from pace.history import *
from pace.plots import *
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
%run 'Information Gain Ratio.ipynb'

In [ ]:
df = pd.read_csv("../../data/Synthetic_APC_DIAG_Fields.csv")

In [ ]:
session = PlotSession(df, session_file="tutorial_example_session.json")

In [ ]:
session.add_plot(name="gaps")

In [ ]:
gaps = session.selected_records("gaps")

print(f"{sum(gaps)} of {len(gaps)} selected records")

In [ ]:
igr_columns = ['ADMIAGE','ADMIMETH','Mortality','PROCODE3','SEX']
igr_result = igr(df[igr_columns], gaps)

igr_result

In [ ]:
plt.bar(igr_result.keys(), igr_result.values())

In [ ]:
session.save("tutorial_example_session.json")